In [ ]:
# Loading csv
health = pd.read_csv(data_path + 'Health_Code_Violations_Seattle.csv')
display(health.head())

In [ ]:
import pandas as pd
import json

def load_jsons(data_path, file):
    """ helper function to load '.json' files (they're not proper jsons) """
    file_path = data_path + file
    with open(file_path) as jsons:
        lines = [json.loads(json_line) for json_line in jsons]
    return lines

In [ ]:
# Load data for town called Ambridge
data_path = './Seattle_Health_Code/'
review_file = 'review.json'
business_file = 'business.json'

reviews = load_jsons(data_path, review_file)
businesses = load_jsons(data_path, business_file)

In [ ]:
rvws = {}
for review in reviews:
    rid = review['business_id']
    if rid in rvws:
        rvws[rid].append(review)
    else:
        rvws[rid] = [review]

In [ ]:
print(rvws['1jyenjdG7kw6i1NEDLr1ZA'])

In [ ]:
# Merging the JSON reviews
bsns = {business['business_id']: business for business in businesses}

for key, rvs in rvws.items():
    bsns[key]['reviews'] = rvs

print(list(bsns.items())[:3])

In [ ]:
#Matching the inspections with reviews
#clean name function
import re
r2 = re.compile(r'[^a-zA-Z ]')

def clean_name(name):
    name1 = r2.sub('', name)
    return name1.lower().strip()

print(clean_name("A!@#112bc"))
print(clean_name("XYZ!*&12778 78#112bc"))

In [ ]:
# Restructure dictionary
bsns_name = {clean_name(value['name']): value for key, value in bsns.items()}
for key, value in bsns_name.items():
    value['inspections'] = []

In [ ]:
print(list(bsns_name.items())[:3])

In [ ]:
# Merging in the inspections
# apply clean_name to name column in health dataframe
health['Name'] = health['Name'].map(clean_name)

In [ ]:
# add inspections to restaurants
for row in health.iterrows():
    name = row[1]['Name']
    business = bsns_name.get(name, None)
    if business:
        business['inspections'].append(row[1].to_dict())

In [ ]:
print(list(bsns_name.items())[:3])